## 创建第一个图谱，然后运行它

In [4]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

f = x * x * y + y + 2

In [5]:
# way1
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)
sess.close()

42


In [6]:
# way2
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [8]:
# way3
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [9]:
# way4
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


## 管理图谱

## 节点值的生命周期

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
    
# 计算y，z时，需要重复求w，x

10
15


In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with Tensorflow

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape

# np.c_按column来组合array
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)



[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


# 实现梯度下降

### 手动计算渐变

In [25]:
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
# np.c_按column来组合array
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)),scaled_housing_data] # 标准化数据

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

# 随机theta
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')

y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta-learning_rate*gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE =  8.1190195
Epoch 100 MSE =  0.96618503
Epoch 200 MSE =  0.75269616
Epoch 300 MSE =  0.68780845
Epoch 400 MSE =  0.6433254
Epoch 500 MSE =  0.61115587
Epoch 600 MSE =  0.58779114
Epoch 700 MSE =  0.57079947
Epoch 800 MSE =  0.5584264
Epoch 900 MSE =  0.5494051
[[ 2.0685523 ]
 [ 0.8179873 ]
 [ 0.16556638]
 [-0.15190868]
 [ 0.17144169]
 [ 0.01267793]
 [-0.04303828]
 [-0.5695357 ]
 [-0.53440386]]


### MINI-BATCH完整代码

In [27]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf 
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape
print('数据集:{}行,{}列'.format(m, n))

housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size)) # ceil()方法返回x的上线 - 不小于x的最小整数

def fetch_batch(epoch, batch_index, batch_size):
#     know = np.random.seed(epoch * batch_index + batch_size)
#     print('我是know:', know)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    
print(best_theta)

数据集:20640行,8列
[[ 2.0589514e+00]
 [ 7.8927094e-01]
 [ 1.1775794e-01]
 [-2.2357766e-01]
 [ 2.8684434e-01]
 [ 2.1559454e-04]
 [-1.7875281e-01]
 [-9.0320015e-01]
 [-8.9115041e-01]]


## 保存和恢复模型
